# Digging into &gamma;-ray bursts

The Burst and Transient Source Experiment ([BATSE](https://gammaray.nsstc.nasa.gov/batse/)) was an instrument aboard the spaceborne Compton Gamma Ray Observatory ([CGRO](https://en.wikipedia.org/wiki/Compton_Gamma_Ray_Observatory)) that flew in low-Earth orbit from 1991 through 2000. BATSE conducted a decade-long, all-sky survey for &gamma;-ray bursts (GRBs) using a set of eight identical sodium iodide (NaI) scintillation detectors aboard the spacecraft. GRBs tend to happen at a rate of about 1/day somewhere on the sky, and can last anywhere from milliseconds to tens of minutes at a time.

This Jupyter notebook examines a population of BATSE GRBs to understand their ensemble properties, which can shed some light (pun shamelessly intended) on the physical mechanisms that power these ultra-relativistic explosions. In the process, we will come across some common data visualization techniques in astronomy and learn how to load data directly over the interwebs without downloading a file.

**Note:** this is strongly related to material from week 4 of the course, themed as  [*In which things get very strange*](https://alurban.github.io/black-hole-astrophysics/week-4/).

## Import useful packages

Recall that Python is an *interpreted* language (rather than a *compiled* one). This means you can interact directly with the Python interpreter to run your code in real-time, without having to compile it. There are many, many, many third-party software applications available through the Python Package Index ([PyPI](https://pypi.org)), which in general can be **imported** so you can use them in your code.

In this example we will make heavy use of [NumPy](https://numpy.org), a fundamental Python package for numerical analysis, and [Astropy](https://www.astropy.org), a common core package for astronomy. We will also make use of Python's built-in `urllib` and `warnings` modules.

In [ ]:
import numpy
import urllib.request
import warnings

# module-level imports
from astropy import (coordinates, units)
from astropy.io import fits

As usual, we will also use [Matplotlib](https://matplotlib.org) and include the rule for rendering figures directly within the notebook.

In [ ]:
# matplotlib imports
import matplotlib.pyplot as plt
from matplotlib import patheffects
%matplotlib inline

## Define global variables

Next we should get in the habit of setting global-scope variables before the main code. Remember that by convention these global variables should always be named in all-caps to distinguish them from other, local-scope variables.

In our case these variables are mostly here to point to the [BATSE 4B catalogue](https://gammaray.nsstc.nasa.gov/batse/grb/catalog/4b/), which has been curated and is available for public use.

In [ ]:
# global variables
CMAP = 'bone_r'
BATSE_PATH = ('https://gammaray.nsstc.nasa.gov/'
              'batse/grb/catalog/current/tables')
BATSE_BASIC = '/'.join((BATSE_PATH, 'basic_table.txt'))
BATSE_FLUX = '/'.join((BATSE_PATH, 'flux_table.txt'))
BATSE_DURATION = '/'.join((BATSE_PATH, 'duration_table.txt'))
HALPHA_SURVEY = ('https://lambda.gsfc.nasa.gov/data/foregrounds/halpha/'
                 'images/lambda_mollweide_halpha_fwhm06_0512.fits')

We will also filter out any `RuntimeWarning` that might come up, since there are likely to be a few division-by-zero warnings we don't really care about.

In [ ]:
warnings.filterwarnings('ignore', category=RuntimeWarning)

## Define a useful helper function

It will prove extremely useful to have a convenience function we can call to fetch BATSE catalogue data and return it to us in a form we can use straight away. BATSE data are hosted as simple tables in `.txt` files, so to retrieve it, we'll need to know the URL to a particular data file and an idea of which table columns are interesting to us. Because of the way one of the tables is stored, with rows broken up across multiple lines, we will also need to be able to parse accordingly.

The algorithm for retrieving BATSE data over the web looks like this:

1. Use the built-in [`urllib.request.urlopen`](https://docs.python.org/3/library/urllib.request.html#urllib.request.urlopen) utility to access a file
2. Use the `readlines()` method for file objects to parse lines given a starting row and a step size (i.e., the number of rows to skip at a time when a table row spans multiple file rows)
3. Re-pack the requested columns as a list of `numpy` arrays using [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions)

**Note:** Python's list indices are *zero-counted*, meaning the first item in the list has index `0`, the second has index `1`, etc. To keep things user-friendly our function will start counting rows at `1`, but we will have to account for this when writing the function.

In [ ]:
# -- functions ----------------------------------------------------------------

def read_txt_from_url(path, columns, start=1, step=1):
    """Reads a .txt file given a URL, and returns the data as a collection of
    numpy arrays

    Parameters
    ----------
    path : `str`
        URL of the file to read

    columns : `tuple` of `int`
        columns to read from the file, should be a `tuple` of `int` for
        multiple columns

    start : `int`, optional
        line of the file to start on, default: 1

    step : `int`, optional
        number of lines to step at a time, default: 1

    Returns
    -------
    data : `list` of `numpy.ndarray`
        requested column data from the file
    """
    # request HTTP access to the given file
    with urllib.request.urlopen(path) as response:
        lines = response.readlines()[start-1::step]
    # re-pack requested rows as a list of arrays
    data = [numpy.array(
        [float(line.split()[i-1]) for line in lines]
    ) for i in columns]
    return data

## Where are they on the sky?

The first basic question we want to answer about these GRBs is: where (that is, what directions) are they coming from? This can be extremely valuable information because the on-sky distribution of sources will look different depending on a few things. I mean, if you think about it:

* Nearby stars are relatively isotropic, meaning they favor no particular direction and are roughly evenly distributed across the sky
* Individual visible stars that are further away, along with certain objects such as pulsars, tend to be clustered together in a (relatively) faint band because they're all in the plane of our home galaxy, the Milky Way
* Separate galaxies and various cosmological objects are once again isotropic, favoring no particular direction over any other

In the first case we're looking at objects closer to us than the thickness of the Milky Way disk, while in the third case, we're looking at objects so far away that they're outside of the Milky Way entirely. Either way this tells us something important about their intrinsic brightness: more distant objects need to be _much_ more luminous in order to be visible to us with the same apparent brightness.

To settle this question for GRBs, we will read out the [**galactic latitude and longitude**](https://en.wikipedia.org/wiki/Galactic_coordinate_system) for each burst, then lay that over the Milky Way and see if they track the shape of our galaxy. We'll first need to grab the GRB positions from [columns 8, 9, and 10](https://gammaray.nsstc.nasa.gov/batse/grb/catalog/4b/4br_basic.html)$^*$ of the BATSE 4B basic catalogue, then take advantage of the `astropy.coordinates` module to get them in easily convertible units.

$^*$*We're reading from columns 8, 9, and 10 rather than columns 7, 8, and 9 because of a whitespace issue in the `.txt` file.*

In [ ]:
# read the galactic longitude, galactic latitude, and error radius
# from columns 8, 9, and 10 of the BATSE 4b basic catalogue
(lon, lat, err) = read_txt_from_url(BATSE_BASIC, columns=(8, 9, 10))
lon = coordinates.Angle((lon + 180) * units.degree)
lat = coordinates.Angle(lat * units.degree)
err = coordinates.Angle(err * units.degree)

# set zero-point of longitude at 180 degrees
lon.wrap_at(180 * units.degree, inplace=True)

We also want to grab some data that visualizes the Milky Way in terms of the &alpha; emission lines of hydrogen, courtesy of the Legacy Archive for Microwave Background Data Analysis ([LAMBDA](https://lambda.gsfc.nasa.gov)). This data is available through the Flexible Image Transport System (FITS) format, a very common way to store astronomical image data. We will use the [`astropy.io.fits`](https://docs.astropy.org/en/stable/io/fits/) module to retrieve it.

**Note:** because `astropy.io.fits.open()` returns an iterable object, we will unpack it directly to a local variable by taking advantage of [Python tuple syntax](https://www.geeksforgeeks.org/unpacking-a-tuple-in-python/). In the use case below, an underscore (`_`) means "dispose of this element because we don't need it."

In [ ]:
# download H-alpha survey data from NASA
with urllib.request.urlopen(HALPHA_SURVEY) as response:
    (_, milkyway) = fits.open(response)

We are now in a position to represent this data on a [Mollweide projection](https://en.wikipedia.org/wiki/Mollweide_projection) of the sky in galactic coordinates. The Milky Way is shown in a white-to-black color scale, with GRB positions shown on top in transparent blue bubbles whose size is proportional to the area of BATSE's positional uncertainty.

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax1 = fig.gca()
ax1.axis('off')

# show a map of the Milky Way
im = ax1.imshow(milkyway.data, origin='lower', cmap=CMAP)
im.set_clim(0, 30)

# draw a new set of axes on top of the Milky Way map
ax2 = fig.add_axes(ax1.get_position(), projection='mollweide', frameon=False)

# draw the location of each BATSE burst, with detector's localization error
ax2.scatter(lon.radian, lat.radian, s=20*err.degree, marker='o',
            alpha=0.4, edgecolor='#222222', facecolor='#4ba6ff')

# make easier-to-read axis labels
labels = ax2.set_xticklabels([
    '330$^{\degree}$', '300$^{\degree}$', '270$^{\degree}$', '240$^{\degree}$',
    '210$^{\degree}$', '180$^{\degree}$', '150$^{\degree}$', '120$^{\degree}$',
    '90$^{\degree}$', '60$^{\degree}$', '30$^{\degree}$'])
for l in labels:
    l.set_path_effects([
        patheffects.Stroke(
            linewidth=4,
            foreground=plt.cm.get_cmap(CMAP)(0)),
        patheffects.Normal()])
plt.show()

Clearly the BATSE GRB population is broadly isotropic: it prefers no direction on the sky, meaning the sources of these bursts are either very nearby or _incredibly_ far away. If the latter, then they must also be absurdly bright, so it may be beneficial for us to investigate their apparent brightness.

## How bright are they?

BATSE also provides the peak apparent brightness as measured by the maximum number of photons incident on the detector per unit area per unit time, a quantity that many of you will probably recognize as [flux](https://youtu.be/VcZe8_RZO8c).

To get a handle on the relevance of this quantity for a population of sources, suppose for a second that every GRB has the same luminosity (or *intrinsic brightness*) and that their sources are distributed uniformly in volume. This would mean that the total number of sources within a certain distance $R$ of Earth is

$$ N(<R) \propto R^3 $$

If $S \propto L/R^2$ is the observed flux, this means the cumulative number of sources brighter than $S$ should be

$$ N(>S) \propto S^{-3/2}. $$

So how does the actual distribution compare?

In [ ]:
# read in BATSE flux data, then sort in ascending order
(flux, ) = read_txt_from_url(
    BATSE_FLUX,
    columns=(1,),
    start=4,
    step=5,
)
flux.sort()

# prepare a figure
fig = plt.figure(figsize=(12, 6))
ax = fig.gca()

# plot a cumulative histogram of log(flux)
plt.hist(flux, bins=flux, histtype='step', color='#4ba6ff',
         linewidth=2, cumulative=-1, label='BATSE 4B catalogue')

# plot the uniform volume distribution
# expected from Euclidean universe
S = numpy.logspace(-1, 3, 101)
N = 10 * (S / 55) ** (-3/2)
ax.plot(S, N, 'k--', linewidth=1.25, label='$S^{-3/2}$')

# x-axis features
ax.set_xscale('log')
ax.set_xlim([1, flux.max()])
ax.set_xlabel('Peak flux [photons cm$^{-2}$ s$^{-1}$]')
ax.set_xticks([1, 10, 100])
ax.set_xticklabels(['1', '10', '100'])

# y-axis features
ax.set_yscale('log')
ax.set_ylim([1, 1e3])
ax.set_ylabel('Number of GRBs')
ax.set_yticks([1, 10, 100, 1000])
ax.set_yticklabels(['1', '10', '100', '$10^3$'])

# draw a legend and show
plt.legend(loc='best')
plt.show()

We get roughly the right shape at high flux but *significant* deviation at smaller fluxes. So what gives?

While this could imply a few possibilities, we now understand that GRBs are *cosmological* in origin, meaning they happen far enough away that the expansion of the universe has to be taken into account when thinking about their volume distribution. But if they can be seen on Earth from clear across the cosmos, this means they have to be unimaginably bright. In fact, &gamma;-ray bursts are the brightest known astrophysical sources - on the electromagnetic spectrum, at least.$^*$ A typical GRB puts out about the same amount of energy over the course of a minute as the Sun will **in its entire lifetime**.

$^*$*The brightest objects ever recorded are binary black hole mergers, but their emission is entirely in the form of gravitational waves, so they aren't "visible" to telescopes in the conventional sense.*

## How long do they last?

Finally, let's take a look at the duration of GRBs. To define this you have to think about the timescale over which 90% of the observed flux was received, centered on the time of the burst. This quantity is commonly referred to as $T_{90}$ and is also available from the BATSE catalogue.

In [ ]:
# read in BATSE duration data
(T90, ) = read_txt_from_url(BATSE_DURATION, columns=(5,))

# prepare a figure
fig = plt.figure(figsize=(12, 6))
ax = fig.gca()

# plot a histogram of log(T90)
bins = numpy.logspace(-3, 3, 31)
ax.hist(T90, bins=bins, histtype='step', color='#222222', linewidth=2)
ax.plot([2, 2], [0, 300], 'k--', linewidth=1)

# x-axis features
ax.set_xscale('log')
ax.set_xlim([1e-3, 1e3])
ax.set_xlabel('$T_{90}$ [seconds]')

# y-axis features
ax.set_ylim([0, 300])
ax.set_ylabel('Number of GRBs')

# show the figure
plt.show()

Interestingly, we find that when we make a histogram of $T_{90}$ and look at it on a logarithmic scale, the distribution we see is *bimodal* (i.e., it has two peaks). Again, this could imply a couple of things, but we now understand that this is because GRBs come in two flavors.

**Long** GRBs generally last for 2 or more seconds and are associated with core-collapse supernovae: as a very massive star reaches the end of its life, it slowly drains of nuclear fuel and eventually collapses to form a black hole. While the black hole is forming, a lot of what used to be the star's outer layers is still laying around, accreting onto the newly-formed black hole in a way that powers the massive intergalactic burst of &gamma;-rays.

Conversely, **short** GRBs last for 2 seconds or less and are associated with compact binary merger events. While this has long been theorized, we now know from observations of the gravitational wave signal [GW170817](https://link.aps.org/doi/10.1103/PhysRevLett.119.161101) that this association is real, and also implies the birth of a black hole (for entirely separate reasons).